#### [ 자연어 전처리 - 정제+토큰화 ]
- 정제 단계 (Cleaning Step)
- 토큰화 단계 (Tokenize Step)

[1] 모듈 로딩 및 데이터 준비<hr>

In [1]:
## 토큰화 관련 모듈
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import WordNetLemmatizer

In [2]:
## 데이터 파일
DATA_FILE   = '../data/test_data.txt'           ## 말뭉치 즉 코퍼스(Copous)     
STOP_WORD   = stopwords.words('english')        ## 불용어 즉, 분석에 의미없는 단어들
PUNCTUATION = punctuation                       ## 구두점


In [3]:
print(f'STOP_WORD   : {len(STOP_WORD)}개\n{STOP_WORD}')
print(f'PUNCTUATION : {len(PUNCTUATION)}개\n{PUNCTUATION}')

STOP_WORD   : 198개
['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'sh

[2] 텍스트 데이터 정제<hr>
- 영어 : 대소문자 일치
- 필요없는 기호, 문자 제거

In [4]:
## - 파일 데이터 읽어오기
with open(DATA_FILE, mode='r', encoding='utf-8') as f:
    fileData=f.read()

##- 확인
print(f'fileData==> {len(fileData)}개')

fileData==> 1851개


In [5]:
## 대소문자 일치 ==> 소문자
fileData=fileData.lower()

##- 확인
print(f'fileData==>\n {fileData[:30]}')

fileData==>
 what a merry-go-round is the e


[3] 문장 단위 토큰화 + 불용어 제거 + 원형 복원 <hr>

In [6]:
## [3-1] 문장 분리 후 문장 단위에서 토큰 분리
sentences = sent_tokenize(fileData)
sentences

['what a merry-go-round is the eighteenth collection by british fashion designer alexander mcqueen, made for the autumn/winter 2001 season of his fashion house alexander mcqueen.',
 "the collection drew on imagery of clowns and carnivals, inspired by mcqueen's feelings about childhood and his experiences in the fashion industry.",
 'the designs were influenced by military chic, cinema such as nosferatu (1922) and cabaret (1972), 1920s flapper fashion, and the french revolution.',
 'the palette comprised dark colours complemented with neutrals and muted greens.',
 'the show marked the first appearance of the skull motif that became a signature of the brand.',
 "the collection's runway show was staged on 21 february 2001 at the gatliff road warehouse in london, as part of london fashion week.",
 "it was mcqueen's final show in london; all his future collections were presented in paris.",
 'sixty-two looks were presented in the main runway show, with at least six more in the finale.',
 '[

In [7]:
## --------------------------------------------------------------
## 함수기능 : 품사 기반 불필요 품사 제거 + 원형 복원 토큰 반환 
## 함수이름 : covertOriginal
## 매개변수 : pos_token_list - (단어, 품사)형태의 토큰 리스트
## 함수결과 : 불필요 품사 제거 + 원형 복원 토큰 리스트
## --------------------------------------------------------------
def covertOriginal(pos_token_list):
    ## 표제어 추출 인스턴스
    wnLemma = WordNetLemmatizer()

    ## 원형 복원 저장 
    result =[]
    print(pos_token_list)
    ## 형용사, 동사 경우 표제어 즉, 원형 복원
    for word, pos in pos_token_list:
            
        ## 형용사, 동사 경우 표제어 즉, 원형 복원
        if pos[:2] in ['JJ', 'VB']:
            result.append(wnLemma.lemmatize(word, 'a' if pos=='JJ' else 'v' ))
        elif pos not in ['DT', 'IN', 'CD', 'CC']:
            ## 토큰 합치기 => 불필요한 품사 제거한 토큰들 그대로 추가
            result.append(word)

    return result

In [21]:
## [3-2] 문장 단위 전처리 된 문장 리스트 추출 

## - 문장별 토큰 리스트, 품사 토큰 리스트 저장 
sent_token_list, pos_token_list =[],[]

## - 문장별 토큰화 처리
for sent in sentences:
    ## 문장 단위로 구두점 제거, 토큰 분리
    for p in PUNCTUATION: 
        if p in sent: sent=sent.replace(p,'')
    
    ## 토큰 분리=> list 반환
    pos_token_list = pos_tag( word_tokenize(sent) )
    print(f'[pos_token_list] ===> {pos_token_list}')

    ## 형용사 'JJ'=> 'a', 동사 'VB' => 'v'원형 복원 
    ## 불필요한 품사 제거한 토큰들 저장
    tokens = covertOriginal(pos_token_list)
    print(f'[tokens] ===> {tokens}')

    ## 토큰 리스트에서 불용어 제거
    words = [ w for w in tokens  if w not in STOP_WORD ]

    ## 문장 단위 토큰 저장 
    sent_token_list.append(words)
    
print(F'총 문장 수 : {len(sent_token_list)}개\n{sent_token_list}')

[pos_token_list] ===> [('what', 'WP'), ('a', 'DT'), ('merrygoround', 'NN'), ('is', 'VBZ'), ('the', 'DT'), ('eighteenth', 'JJ'), ('collection', 'NN'), ('by', 'IN'), ('british', 'JJ'), ('fashion', 'NN'), ('designer', 'NN'), ('alexander', 'NN'), ('mcqueen', 'NN'), ('made', 'VBN'), ('for', 'IN'), ('the', 'DT'), ('autumnwinter', 'NN'), ('2001', 'CD'), ('season', 'NN'), ('of', 'IN'), ('his', 'PRP$'), ('fashion', 'NN'), ('house', 'NN'), ('alexander', 'NN'), ('mcqueen', 'NN')]
[('what', 'WP'), ('a', 'DT'), ('merrygoround', 'NN'), ('is', 'VBZ'), ('the', 'DT'), ('eighteenth', 'JJ'), ('collection', 'NN'), ('by', 'IN'), ('british', 'JJ'), ('fashion', 'NN'), ('designer', 'NN'), ('alexander', 'NN'), ('mcqueen', 'NN'), ('made', 'VBN'), ('for', 'IN'), ('the', 'DT'), ('autumnwinter', 'NN'), ('2001', 'CD'), ('season', 'NN'), ('of', 'IN'), ('his', 'PRP$'), ('fashion', 'NN'), ('house', 'NN'), ('alexander', 'NN'), ('mcqueen', 'NN')]
[tokens] ===> ['what', 'merrygoround', 'be', 'eighteenth', 'collection', '

[4] 단어 사전 생성 <hr>
- 토큰 -- 정수 인코딩
- 특수토큰 : 없는 토큰 UNK, 길이 맞춤용 토큰 PAD

(4-1) 중복 단어 제거

In [9]:

## - 문장의 토큰들 하나로 합치기
token_list = [ word  for sent in sent_token_list  for word in sent  ]

print(f'token_list => {token_list}')
print(f'token_list => {len(token_list)}개')

## - 토큰들 중복 제거
token_list = list(set(token_list))
print(f'token_list => {token_list}')
print(f'token_list => {len(token_list)}개')

token_list => ['merrygoround', 'eighteenth', 'collection', 'british', 'fashion', 'designer', 'alexander', 'mcqueen', 'make', 'autumnwinter', 'season', 'fashion', 'house', 'alexander', 'mcqueen', 'collection', 'draw', 'imagery', 'clowns', 'carnivals', 'inspire', 'mcqueens', 'feelings', 'childhood', 'experiences', 'fashion', 'industry', 'designs', 'influence', 'military', 'chic', 'cinema', 'nosferatu', 'cabaret', 'flapper', 'fashion', 'french', 'revolution', 'palette', 'comprise', 'dark', 'colours', 'complement', 'neutrals', 'mute', 'greens', 'show', 'mark', 'first', 'appearance', 'skull', 'motif', 'become', 'signature', 'brand', 'collections', 'runway', 'show', 'stag', 'february', 'gatliff', 'road', 'warehouse', 'london', 'part', 'london', 'fashion', 'week', 'mcqueens', 'final', 'show', 'london', 'future', 'collections', 'present', 'paris', 'sixtytwo', 'looks', 'present', 'main', 'runway', 'show', 'least', 'finale', 'show', 'stag', 'dark', 'room', 'carousel', 'centre', 'finale', 'lights

(4-2) 단어 사전 생성<hr>

In [10]:
### dict 타입으로 단어사전 생성
VOCAB_TO_IDX = {'<PAD>':0, '<UNK>':1 }
VOCAB_TO_IDX

{'<PAD>': 0, '<UNK>': 1}

In [11]:
## 토큰들에게 정수 숫자 부여
for idx, token in enumerate(token_list, 2):
    VOCAB_TO_IDX[token]=idx 

## idx => word로 변환
IDX_TO_TOKEN = {v:k for k,v in VOCAB_TO_IDX.items()}


In [12]:

print('VOCAB_TO_IDX=>\n', VOCAB_TO_IDX)
print('IDX_TO_TOKEN=>\n', IDX_TO_TOKEN)

VOCAB_TO_IDX=>
 {'<PAD>': 0, '<UNK>': 1, 'attract': 2, 'toxic': 3, 'cabaret': 4, 'imagery': 5, 'ensembles': 6, 'become': 7, 'collections': 8, 'skull': 9, 'rear': 10, 'elements': 11, 'conglomerate': 12, 'models': 13, 'looks': 14, 'childhood': 15, 'stage': 16, 'springsummer': 17, 'greens': 18, 'room': 19, 'autumnwinter': 20, 'colours': 21, 'generally': 22, 'mcqueens': 23, 'sixtytwo': 24, 'pose': 25, 'comprise': 26, 'suffocating': 27, 'appearance': 28, 'runway': 29, 'discarded': 30, 'nosferatu': 31, 'exhibitions': 32, 'fashion': 33, 'eveningwear': 34, 'complement': 35, 'evil': 36, 'serve': 37, 'messaging': 38, 'positive': 39, 'reveal': 40, 'piles': 41, 'stag': 42, 'influence': 43, 'house': 44, 'mark': 45, 'french': 46, 'neutrals': 47, 'road': 48, 'appear': 49, 'theme': 50, 'beauty': 51, 'revolution': 52, 'part': 53, 'warehouse': 54, 'flapper': 55, 'turbulent': 56, 'authors': 57, 'clowns': 58, 'come': 59, 'palette': 60, 'experiences': 61, 'merrygoround': 62, 'academic': 63, 'carnivals': 64

[5] 자연어 => 숫자 변환 <hr>

In [13]:
## 문장별 토큰 ==> 수치화 
SENT_NUM_LIST =[]
for sent_line in sent_token_list:
    setnums= [ VOCAB_TO_IDX[word] for word in sent_line]
    print('\n[문장]', sent_line)
    print('[수치]', setnums)
    SENT_NUM_LIST.append(setnums)



[문장] ['merrygoround', 'eighteenth', 'collection', 'british', 'fashion', 'designer', 'alexander', 'mcqueen', 'make', 'autumnwinter', 'season', 'fashion', 'house', 'alexander', 'mcqueen']
[수치] [62, 84, 101, 96, 33, 114, 69, 109, 82, 20, 79, 33, 44, 69, 109]

[문장] ['collection', 'draw', 'imagery', 'clowns', 'carnivals', 'inspire', 'mcqueens', 'feelings', 'childhood', 'experiences', 'fashion', 'industry']
[수치] [101, 86, 5, 58, 64, 100, 23, 113, 15, 61, 33, 94]

[문장] ['designs', 'influence', 'military', 'chic', 'cinema', 'nosferatu', 'cabaret', 'flapper', 'fashion', 'french', 'revolution']
[수치] [83, 43, 90, 102, 104, 31, 4, 55, 33, 46, 52]

[문장] ['palette', 'comprise', 'dark', 'colours', 'complement', 'neutrals', 'mute', 'greens']
[수치] [60, 26, 78, 21, 35, 47, 66, 18]

[문장] ['show', 'mark', 'first', 'appearance', 'skull', 'motif', 'become', 'signature', 'brand']
[수치] [107, 45, 87, 28, 9, 120, 7, 88, 121]

[문장] ['collections', 'runway', 'show', 'stag', 'february', 'gatliff', 'road', 'wareho

In [14]:
## 숫자 문장들 체크
for _ in SENT_NUM_LIST: print( _ )

[62, 84, 101, 96, 33, 114, 69, 109, 82, 20, 79, 33, 44, 69, 109]
[101, 86, 5, 58, 64, 100, 23, 113, 15, 61, 33, 94]
[83, 43, 90, 102, 104, 31, 4, 55, 33, 46, 52]
[60, 26, 78, 21, 35, 47, 66, 18]
[107, 45, 87, 28, 9, 120, 7, 88, 121]
[8, 29, 107, 42, 68, 110, 48, 54, 95, 53, 95, 33, 81]
[23, 92, 107, 95, 67, 8, 80, 76]
[24, 14, 80, 122, 29, 107, 71, 65]
[107, 42, 78, 19, 89, 91]
[65, 93, 59, 40, 41, 30, 15, 106, 10, 16, 13, 85, 36, 58, 118, 16, 25, 34]
[77, 117, 101, 22, 39, 2, 63, 111, 50, 38]
[23, 98, 107, 17, 62, 37, 74, 33, 94, 72, 119, 3, 27]
[116, 11, 97, 57, 103, 99, 46, 73, 108, 12, 112, 115, 109, 56, 75]
[6, 62, 49, 32, 109, 105, 69, 109, 70, 51]


[6] 패딩 : 모든 문장 길이 맞추기<hr>

In [15]:
## 문자별 단어 갯수 체크
length  = [ len(sent) for sent in SENT_NUM_LIST]

print(f'문장별 단어 개수 :  {length}')
print(f'가장 긴 문장    :  {max(length)}, 가장 짧은 문장 :{min(length)}')

문장별 단어 개수 :  [15, 12, 11, 8, 9, 13, 8, 8, 6, 18, 10, 13, 15, 10]
가장 긴 문장    :  18, 가장 짧은 문장 :6


In [16]:
## 문장별 길이 일치 => 가장 긴문장
MAX_LEN = max(length)
NUMS = len(SENT_NUM_LIST)
for idx in range(NUMS):
    sent_len = len(SENT_NUM_LIST[idx])
    if sent_len != MAX_LEN:
        for _ in range(MAX_LEN-sent_len):
            SENT_NUM_LIST[idx].append( VOCAB_TO_IDX['<PAD>'])

In [17]:
for _ in SENT_NUM_LIST:
    print(f'{len(_)}개 : {_}')

18개 : [62, 84, 101, 96, 33, 114, 69, 109, 82, 20, 79, 33, 44, 69, 109, 0, 0, 0]
18개 : [101, 86, 5, 58, 64, 100, 23, 113, 15, 61, 33, 94, 0, 0, 0, 0, 0, 0]
18개 : [83, 43, 90, 102, 104, 31, 4, 55, 33, 46, 52, 0, 0, 0, 0, 0, 0, 0]
18개 : [60, 26, 78, 21, 35, 47, 66, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
18개 : [107, 45, 87, 28, 9, 120, 7, 88, 121, 0, 0, 0, 0, 0, 0, 0, 0, 0]
18개 : [8, 29, 107, 42, 68, 110, 48, 54, 95, 53, 95, 33, 81, 0, 0, 0, 0, 0]
18개 : [23, 92, 107, 95, 67, 8, 80, 76, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
18개 : [24, 14, 80, 122, 29, 107, 71, 65, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
18개 : [107, 42, 78, 19, 89, 91, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
18개 : [65, 93, 59, 40, 41, 30, 15, 106, 10, 16, 13, 85, 36, 58, 118, 16, 25, 34]
18개 : [77, 117, 101, 22, 39, 2, 63, 111, 50, 38, 0, 0, 0, 0, 0, 0, 0, 0]
18개 : [23, 98, 107, 17, 62, 37, 74, 33, 94, 72, 119, 3, 27, 0, 0, 0, 0, 0]
18개 : [116, 11, 97, 57, 103, 99, 46, 73, 108, 12, 112, 115, 109, 56, 75, 0, 0, 0]
18개 : [6, 62, 49, 32, 109, 105, 69, 10

[7] 인코딩<hr>

In [24]:
import numpy as np 
from sklearn.preprocessing import OneHotEncoder 

dataNP = np.array(SENT_NUM_LIST)
dataNP.shape

(14, 18)

In [25]:
## 인코딩
ohEncoder = OneHotEncoder(sparse_output=False)
SENT_VEC=ohEncoder.fit_transform(dataNP)

In [26]:
SENT_VEC

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 1., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.]])